<a href="https://colab.research.google.com/github/desik98/DATA-MANAGEMENT-AND-DATA-SYSTEMS-BY-STANFORD/blob/master/github_database_design_and_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS145: Project 2
## Part 2 | Exploring the GitHub Dataset with Colaboratory (60 points)
---

### Notes (read carefully!):

* Be sure you read the instructions on each cell and understand what it is doing before running it.
* Don't forget that if you can always re-download the starter notebook from the course website if you need to.
* You may create new cells to use for testing, debugging, exploring, etc., and this is in fact encouraged!
**Just make sure that the final answer for each question is _in its own cell_ and _clearly indicated_**.
* Colab will not warn you about how many bytes your SQL query will consume.  **Be sure to check on the BigQuery UI first before running queries here!**
* See the assignment handout for submission instructions.
* Have fun!

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Name, SUNet ID*

## Setting Up BigQuery and Dependencies

Run the two cells below (shift + enter) to authenticate your project and import the libraries you'll need. 

Note that you need to fill in the `project_id` variable with the Google Cloud project id you are using for this course.  You can see your project ID by going to https://console.cloud.google.com/cloud-resource-manager.



In [0]:
# Run this cell to authenticate yourself to BigQuery.
from google.colab import auth
auth.authenticate_user()
project_id = "my-project-28530"

In [0]:
# Some imports you will need
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
plt.style.use('seaborn-whitegrid')

# Overview

As we saw in class, BigQuery has a massive dataset of GitHub files and statistics, including information about repositories, commits, and file contents.  In this part of the assignment, we will be working with this dataset further.  Don't worry if you are not too familiar with Git and GitHub -- we will explain everything you need to know to complete this part of the assignment.

---
## Notes

**The GitHub dataset available on BigQuery is actually quite massive. *A single query on the "contents" table alone (it is 2.16TB!) can eat up your 1TB allowance for the month AND cut into about 10% of your GCloud credit for the class.***

To make this part of the project more manageable, we have subset the original data.  We have preserved almost all information in the original tables, but we kept only the information on the top 500,000 most "watched" GitHub repos between January, 2016 and the most recent data as of October, 2018.

You can see the tables we will be working with [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2). **Read through the schemas to get familiar with the data**.  Note that some of the tables are still quite large (the contents table is about 500GB), so you should exercise the usual caution when working with them.  Before running queries on this notebook, it's good practice to first set up query limits on your BigQuery account or see how many bytes will be billed on the web UI.

---

## A Super Quick Primer on Git

If you are not very familiar with Git and GitHub, here are some high-level explanations that will give you enough context to get you through this part of the problem:

*  *GitHub*: GitHub is a source-control service provider.  GitHub allows you to collaborate on and keep track of source code in a fairly efficient way.

*  *commit*: A commit can be thought of as a change that is applied to some set of files. i.e., if some set of files is in some state A, you can make changes to A and *commit* your changes to the set of files so that it is now in state B.  A commit is identified by a *hash* of the information in your change (the author of the commit, who actually committed [i.e. applied] the changes to the set of files, the changes themselves, etc.)

* *parent commit*: The commit that came before your current commit.

*   *repo*:  A repo (short for repository) is GitHub's abstraction for a collection of files along with a history of commits on those files. If you have GitHub username "foo" and you make a repository called "data-rocks", your repo name will be "foo/data-rocks".  You can think of a repo's history in terms of its commits.  E.g., "foo/data-rocks" can have the set of "states" A->B->C->D, where each state change (A->B, B->C, C->D) was due to a commit.

*   *branch*:  To keep track of different commit histories, GitHub repos can have branches.  The 'main' branch (i.e. commit history) of the repo is called the 'master' branch.  Say on "foo/data-rocks" we have the commit history A->B->C->D on the master branch.  If someone else comes along and decides to add a cool new feature to "foo/data-rocks", they can create a branch called "cool-new-feature" that branches away from the master branch. All the code from the main branch will be there, but new code added to "cool-new-feature" will not be on the main branch.

*   *ref*: For the purpose of this assignment, you can think of the 'ref' field on the "files" table as referring to the branch in which a file lives in a repository at some point in time.



For the purposes of this question, you don't need to know about the following things in detail:


*   Commit trees
*   The encoding attribute on the commits table

If you want more clarifications about Git and GitHub in order to answer this question, be sure to post on Piazza or come to Office Hours.  In many cases, a quick web search will also help answer your question.

# Section 1 | Getting Familiar with the GitHub Data
---

## Understanding the GitHub Tables

As you saw in lecture, E/R diagrams are a way to convey the structure of a database (a set of tables) in a non-technical way that is still precise enough for us to work with technically.  As it turns out, E/R diagrams will prove to be a useful tool in getting familiar with the structure of our GitHub tables before we even analyze any of its data!

## Question 1: CS145 GitHub tables -->  E/R Diagram (6 points)

Using what you learned in lecture, create an E/R diagram for the data contained in the `cs145-ta:project_part2 github_repo` tables [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2).  For the purposes of this exercise, ignore the `github_repo_readme_contents_cs145` table (we'll be using that one later)

### Notes

* The CS145 GitHub Repo tables may not directly translate into an E/R diagram as described in class.  An important part of this question is for you to analyze the existing tables, think about the relationships between objects in the tables as a whole, and construct a reasonable E/R diagram from that analysis.

* Consider the "author" and "committer" attribute structures a single attribute each.  Importantly, you do not need to include `committer.name`, `committer.email`, etc.  in your E/R diagrams.  Note that the "language" attribute is actually an *array* type. Be sure to account for this in your E/R diagram.

* Your E/R diagram should be an image.  You can hand draw it or generate it with your favorite software.  You can include an image in your Colab Notebook as follows:



> 1. Add the image to your own Google Drive.  
> 2. Get a shareable link for your image, and note the "ID" field of the URL.
> > Your link will look something like "`https://drive.google.com/open?id=<some ID>`"
> 3. Add the following markup to your cell `![](https://drive.google.com/uc?export=view&id=<your image ID>)`
> 4. Run the cell


![Github Database ER Diagram](https://drive.google.com/open?id=1eAfdfBz7LtqbaE-9VsE9vsM-ob70QVEN)

## Question 2: Explain your E/R Diagram (4 points)

In a small paragraph, explain your E/R diagram.  You should cover at least:

*   what your entities are
*   what the relationships between those entities are (including whether they are one-to-many, many-to-one, etc.)
*   what your keys are for each entity

You should also include a quick explanation of how you determined the overall structure of your chart.

---
# Entities
*   users
*   repositories
*   repo_files
*   repo_commits
*   repo_changes

# Relationships
*   Users do **changes** via commits.
*   repo_file belongs to repositories (Many to One)
*   commits belongs to repositories (Many to One)

# Keys
     
     Primary Key : PK
     Foreign Key : FK
     
*   users:  PK:  user_id
        
*   repositories: repo_id
        
*   repo_files: file_id, repo_id
        
*   repo_commits: commit_id, repo_id
        
*   repo_changes: (commit_id + user_id), (commit_id, user_id)
---

## Question 3: Translate your E/R Diagram to a Set of Schemas (5 points)

Provide a set of schemas that translate directly to the E/R Diagram you generated in Question 2.  These schemas should not be the same as the schemas in the CS145 GitHub data tables. 

Be sure to specify in your schemas:
1. the ***name*** of each attribute (you don't need to worry about types)
2. the ***key for each schema***
3. ***Foreign keys in your schemas*** and what table they reference. 

---


# users

   user_id | user_name | user_details
----------------|-----------------|-----------------
    1	 |   Linus Torvalds   |   Torvald_Details
   

# Repositories

   repo_id | repo_name | repo_licenses | repo_watch_count
----------------|-----------------|-----------------|-----------------|-----------------
    1	 |   torvalds/linux |  mit | 44121 
    
# Repo_Languages

   repo_id | repo_languages
----------------|-----------------
    1	 |   C

# Repo_Files

   file_id | file_name | repo_id | file_path | file_size | file_content | file_author_id
----------------|-----------------|-----------------|----------------|-----------------|-----------------|
    1	 |  9a4d9adb7b8c7c73820aaa73cf4f491925d013d2	  |  sound/core/seq/oss/seq_oss_event.h | spotify/linux | 2800 |  `/*code*/`| Linus Torvalds
    
# Repo_Commits

   commit_id | commit_name | commit_message | repo_id
----------------|-----------------|-----------------|-----------------|-----------------
    1	 |  0dc831b9bca98a0d8dafb00fa7f20b3aef6cab67 |  commited | 1 |
    
# changes

   commit_id | user_id | commit_date
----------------|-----------------|-----------------
    1	 |  1  |  01-04-2019 UTC
   
---

## Question 4: Analysis (4 points)

You now have two sets of schemas:  the ones you came up with in Question 3 based on your E/R diagram, and those that we provided for you.  

**In a single paragraph (no more than 100 words), comment on the tradeoffs between the two.  Which do you think is better?**

Use the discussions that we have had in class as a starting point for your analysis.   There is no single correct answer to this question.  Some people abide strongly to their E/R diagrams, while others decide to slice and dice what would have been on an E/R diagram to better fit their needs.


---
  # Design Comparison
  
      1. Language Attribute in Repo_languages is not atomic, this violates Normalization.
      2. Redundancy:  Author, Commiter, Repo_Name in github_repo_commits_cs145 might be redundant.
      3. Author needs to be a seperate entity.
      4. Repo_contents and repo_files can be merged.
      5. repo_languages can be made atomic.
      6. github_repo_licenses_cs145, github_repo_readme_contents_cs145, github_repos_cs145 can be merged. 
      7. And all the primary keys can be integers instead of strings.
      
---

# Section 2 | Git Visualizing!
---

## Initial Exploration

Now that we understand the dataset we will be working with very well, let's get down to analyzing some of its properties. For the visualizations in this section, feel free to use whatever graphing library you are most comfortable with.  We recommend using one of the following:


*   seaborn (https://seaborn.pydata.org/tutorial.html)
*   matplotlib (https://matplotlib.org/3.0.0/tutorials/index.html)
*   Altair (https://altair-viz.github.io/)
*   Pandas (https://pandas.pydata.org/pandas-docs/stable/visualization.html)


> * **Note**: that you can plot directly from a Pandas DataFrame if you'd like (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)!





### How to BigQuery in Colab

Jupyter notebooks (what Colab notebooks are based on) have a concept called "magics".
If you write the following line at the top of a `Code` cell:

```
%%bigquery --project $project_id variable # this is the key line
SELECT ....
FROM ...
```

That "%%" converts the cell into a SQL cell. The resulting table that is generated is saved into `variable`. You can now use `variable` with your favorite visualization library to generate plots!

Run the following two cells to see this in action.

In [0]:
%%bigquery --project my-project-28530 example

SELECT lrepo_name, watch_count
FROM `cs145-ta.project_part2.github_repos_cs145`
ORDER BY watch_count DESC
LIMIT 20

,lrepo_name,watch_count
0,freecodecamp/freecodecamp,291503
1,vuejs/vue,119634
2,tensorflow/tensorflow,107721
3,facebook/react,92644
4,sindresorhus/awesome,73781
5,getify/you-dont-know-js,71632
6,kamranahmedse/developer-roadmap,59674
7,microsoft/vscode,57390
8,airbnb/javascript,55436
9,twbs/bootstrap,52244


In [0]:
example.head()

,lrepo_name,watch_count
0,freecodecamp/freecodecamp,291503
1,vuejs/vue,119634
2,tensorflow/tensorflow,107721
3,facebook/react,92644
4,sindresorhus/awesome,73781


## Question 5: Count Breakdowns Across Various Fields (9 points)



Let's get out feet wet with this data by creating the following plots:

1.   License distribution across repos
2.   Language distribution across repos
3.   File size distribution across repos
4.   Distribution of files associated with a repo
5.   Number of commits by author and committer across repos
6.   The distribution of the length of commit messages across repos


Note that you will not receive full credit if your charts are  poorly made (i.e. very unclear or unreadable).


### Hints
* Some of these plots will need at least one of their axes to be log-scaled in order to be readable
* For more readable plots, you can use [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html).  A sample size between 1,000 and 10,000 should give you more readable plots.

##  Reminders
* Be careful with your queries!  Don't run `SELECT *` blindly on a table in this Colab notebook since you will not get a warning of how much data the query will consume.  Always how much data a query will consume on the BigQuery UI first -- you are also better off setting a query limit as we described earlier. 
* Don't forget to use the subsetted GitHub tables we provide [here](https://bigquery.cloud.google.com/dataset/cs145-ta:project_part2), not the original ones on BigQuery. 




### a) License distribution (1 point)
#### (x-axis: license type, y-axis: # repos containing that license)


In [0]:
%%bigquery --project my-project-28530 license_distribution

SELECT COUNT(lrepo_name) as license_count, license
FROM `cs145-ta.project_part2.github_repo_licenses_cs145`
GROUP BY license
ORDER BY license_count DESC

,license_count,license
0,99730,mit
1,28578,apache-2.0
2,14906,gpl-3.0
3,8748,bsd-3-clause
4,8673,gpl-2.0
5,3692,bsd-2-clause
6,2204,agpl-3.0
7,2057,lgpl-3.0
8,1498,unlicense
9,1461,isc


In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
from altair import *
import pandas as pd

Chart(license_distribution).mark_bar ().encode(
 x = X('license'),
 y = Y('license_count')
)

Chart({
  data:     license_count       license
  0           99730           mit
  1           28578    apache-2.0
  2           14906       gpl-3.0
  3            8748  bsd-3-clause
  4            8673       gpl-2.0
  5            3692  bsd-2-clause
  6            2204      agpl-3.0
  7            2057      lgpl-3.0
  8            1498     unlicense
  9            1461           isc
  10           1200      lgpl-2.1
  11           1197       mpl-2.0
  12           1030       cc0-1.0
  13            852       epl-1.0
  14            153  artistic-2.0,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'license'
    }),
    y: Y({
      shorthand: 'license_count'
    })
  }),
  mark: 'bar'
})

### b) Language distribution (1 point)
#### (x-axis: programming language, y-axis: # repos containing at least one file in that language)

To keep the chart readable, only keep the top 20 languages.

**Hint:** https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays


In [0]:
%%bigquery --project my-project-28530 language_distribution

SELECT repo_language, COUNT(repo_language) as language_usage_count
FROM (SELECT lrepo_name, language_unnested.name as repo_language
      FROM `cs145-ta.project_part2.github_repo_languages_cs145`
      CROSS JOIN UNNEST(language) as language_unnested)
GROUP BY repo_language
ORDER BY language_usage_count DESC
LIMIT 20

,repo_language,language_usage_count
0,JavaScript,60436
1,Shell,45237
2,HTML,42574
3,Python,36700
4,CSS,35450
5,Makefile,22454
6,Java,21377
7,Ruby,19881
8,C,19390
9,C++,17522


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import pandas as pd

Chart(language_distribution).mark_bar ().encode(
 x = X('repo_language'),
 y = Y('language_usage_count')
)

Chart({
  data:    repo_language  language_usage_count
  0     JavaScript                 60436
  1          Shell                 45237
  2           HTML                 42574
  3         Python                 36700
  4            CSS                 35450
  5       Makefile                 22454
  6           Java                 21377
  7           Ruby                 19881
  8              C                 19390
  9            C++                 17522
  10   Objective-C                 13860
  11           PHP                 13433
  12            Go                  9580
  13            C#                  8560
  14     Batchfile                  8078
  15         Swift                  5966
  16         CMake                  5460
  17    TypeScript                  4483
  18    Dockerfile                  4261
  19          Perl                  4012,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'repo_language'
    }),
    y: Y({
      shorthand: 'language_usage_count'
    })
  }),
  mark: 'bar'
})

### c) File size distribution (1 point)
#### (x-axis: file size, y-axis: # files of that size) 

In [0]:
%%bigquery --project my-project-28530 file_size_distribution

SELECT size
FROM `cs145-ta.project_part2.github_repo_contents_cs145`

In [0]:
# YOUR PLOT CODE HERE



### d) Distribution of files associated with a repo (1 point)
#### (x-axis: # files associated with a repo, y-axis: # repos associated with that many files)

In [0]:
%%bigquery --project my-project-28530 file_size_distribution

SELECT COUNT(total_repo_files) as total_repo_total_repo_files, total_repo_files
FROM (SELECT COUNT(id) as total_repo_files
      FROM `cs145-ta.project_part2.github_repo_files_cs145` repo_files_count
      GROUP BY lrepo_name)
GROUP BY total_repo_files
ORDER BY total_repo_total_repo_files DESC
LIMIT 100

,total_repo_total_repo_files,total_repo_files
0,3311,9
1,3302,10
2,3280,11
3,3178,8
4,3082,12
5,2958,7
6,2949,14
7,2887,13
8,2875,6
9,2789,15


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import pandas as pd

Chart(file_size_distribution).mark_bar().encode(
                  x = X('total_repo_files'), 
                  y = Y('total_repo_total_repo_files'),
            )

Chart({
  data:     total_repo_total_repo_files  total_repo_files
  0                          3311                 9
  1                          3302                10
  2                          3280                11
  3                          3178                 8
  4                          3082                12
  5                          2958                 7
  6                          2949                14
  7                          2887                13
  8                          2875                 6
  9                          2789                15
  10                         2739                16
  11                         2721                 5
  12                         2585                17
  13                         2504                18
  14                         2372                19
  15                         2347                20
  16                         2246                 4
  17                         2204                21
  18                         2156                22
  19                         2043                23
  20                         1962                25
  21                         1948                24
  22                         1930                 3
  23                         1805                26
  24                         1731                27
  25                         1711                28
  26                         1677                29
  27                         1534                30
  28                         1533                31
  29                         1496                33
  ..                          ...               ...
  70                          579                71
  71                          570                75
  72                          567                73
  73                          556                74
  74                          513                80
  75                          512                82
  76                          511                77
  77                          508                76
  78                          506                78
  79                          496                79
  80                          492                81
  81                          487                84
  82                          478                87
  83                          470                85
  84                          457                83
  85                          426                 1
  86                          426                86
  87                          415                88
  88                          405                89
  89                          403                91
  90                          394               101
  91                          393                90
  92                          382                94
  93                          372                95
  94                          371                96
  95                          371               100
  96                          370                92
  97                          363                98
  98                          358                93
  99                          351                97
  
  [100 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'total_repo_files'
    }),
    y: Y({
      shorthand: 'total_repo_total_repo_files'
    })
  }),
  mark: 'bar'
})

### e) Number of commits by author and committer (1 point)
#### (x-axis: # commits, y-axis: # authors/committers with that many commits)

**Note**: for this chart, plot the curves for author and committers side-by-side for comparison.



In [0]:
%%bigquery --project my-project-28530 commiter_commit_frequency

SELECT count_commit, COUNT(count_commit) as count_commiter_count
FROM (SELECT committer.name, COUNT(DISTINCT(commit)) as count_commit
      FROM `cs145-ta.project_part2.github_repo_commits_cs145`
      WHERE committer.name != author.name
      GROUP BY committer.name)
GROUP BY count_commit
ORDER BY count_commiter_count DESC, count_commit
LIMIT 100

,count_commit,count_commiter_count
0,1,25507
1,2,10423
2,3,6243
3,4,4441
4,5,3357
5,6,2720
6,7,2227
7,8,1938
8,9,1665
9,10,1395


In [0]:
%%bigquery --project my-project-28530 author_commit_frequency

SELECT count_commit, COUNT(count_commit) as count_commiter_count
FROM (SELECT committer.name, COUNT(DISTINCT(commit)) as count_commit
      FROM `cs145-ta.project_part2.github_repo_commits_cs145`
      WHERE committer.name = author.name
      GROUP BY committer.name)
GROUP BY count_commit
ORDER BY count_commiter_count DESC, count_commit
LIMIT 100

,count_commit,count_commiter_count
0,1,167196
1,2,76635
2,3,46735
3,4,33161
4,5,24364
5,6,19839
6,7,15798
7,8,13403
8,9,11370
9,10,9923


In [0]:
# YOUR PLOT CODE HERE - AUTHORS

import altair as alt
import pandas as pd

Chart(author_commit_frequency).mark_bar().encode(
                  x = X('count_commit'), 
                  y = Y('count_commiter_count')
)

Chart({
  data:     count_commit  count_commiter_count
  0              1                167196
  1              2                 76635
  2              3                 46735
  3              4                 33161
  4              5                 24364
  5              6                 19839
  6              7                 15798
  7              8                 13403
  8              9                 11370
  9             10                  9923
  10            11                  8782
  11            12                  8035
  12            13                  7196
  13            14                  6440
  14            15                  6027
  15            16                  5522
  16            17                  5058
  17            18                  4725
  18            19                  4235
  19            20                  4103
  20            21                  3893
  21            22                  3646
  22            23                  3435
  23            24                  3355
  24            25                  2979
  25            26                  2934
  26            27                  2810
  27            28                  2666
  28            29                  2589
  29            30                  2459
  ..           ...                   ...
  70            71                   831
  71            76                   824
  72            77                   801
  73            73                   789
  74            75                   788
  75            78                   777
  76            72                   773
  77            74                   764
  78            80                   739
  79            81                   736
  80            86                   717
  81            83                   698
  82            84                   698
  83            79                   694
  84            82                   675
  85            85                   641
  86            88                   631
  87            87                   627
  88            90                   626
  89            91                   626
  90            92                   611
  91            89                   582
  92            96                   568
  93            94                   567
  94           100                   559
  95            97                   556
  96            98                   553
  97            93                   543
  98            95                   535
  99           101                   523
  
  [100 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'count_commit'
    }),
    y: Y({
      shorthand: 'count_commiter_count'
    })
  }),
  mark: 'bar'
})

In [0]:
# YOUR PLOT CODE HERE - COMMITTERS

import altair as alt
import pandas as pd

Chart(commiter_commit_frequency).mark_bar().encode(
                  x = X('count_commit'), 
                  y = Y('count_commiter_count')
)

Chart({
  data:     count_commit  count_commiter_count
  0              1                 25507
  1              2                 10423
  2              3                  6243
  3              4                  4441
  4              5                  3357
  5              6                  2720
  6              7                  2227
  7              8                  1938
  8              9                  1665
  9             10                  1395
  10            11                  1257
  11            12                  1176
  12            13                   976
  13            14                   923
  14            15                   849
  15            16                   787
  16            17                   712
  17            18                   666
  18            19                   643
  19            20                   553
  20            21                   536
  21            22                   521
  22            24                   465
  23            23                   462
  24            25                   406
  25            26                   398
  26            28                   376
  27            27                   363
  28            31                   354
  29            29                   339
  ..           ...                   ...
  70            71                    93
  71            80                    82
  72            73                    80
  73            72                    79
  74            76                    78
  75            78                    78
  76            82                    78
  77            77                    76
  78            81                    74
  79            89                    72
  80            74                    71
  81            75                    70
  82            88                    67
  83            85                    66
  84            83                    65
  85            84                    65
  86            79                    64
  87            86                    64
  88            90                    63
  89            98                    63
  90            91                    58
  91            96                    58
  92            92                    57
  93            95                    57
  94            93                    55
  95            94                    55
  96           101                    55
  97           102                    55
  98           106                    55
  99            87                    53
  
  [100 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'count_commit'
    }),
    y: Y({
      shorthand: 'count_commiter_count'
    })
  }),
  mark: 'bar'
})

### f) The distribution of the length of commit messages (1 point)
#### (x-axis: length of the commit message, y-axis: # commits with that length)

**Note**: The query for this plot may use ~30GB of data.

In [0]:
%%bigquery --project my-project-28530 commit_message_length_distribution

SELECT commit_message_length, COUNT(commit_message_length) as commit_message_length_count
FROM (SELECT commit, AVG(LENGTH(message)) as commit_message_length
      FROM `cs145-ta.project_part2.github_repo_commits_cs145`
      GROUP BY commit)
GROUP BY commit_message_length
ORDER BY commit_message_length_count DESC
LIMIT 1000

,commit_message_length,commit_message_length_count
0,16.0,1372101
1,37.0,1087326
2,26.0,969916
3,22.0,959003
4,23.0,923422
5,29.0,910615
6,34.0,909333
7,20.0,908799
8,30.0,907506
9,17.0,902384


In [0]:
# YOUR PLOT CODE HERE

import altair as alt
import pandas as pd

Chart(commit_message_length_distribution).mark_bar(clip = True).encode(
                  x = X('commit_message_length', scale=alt.Scale(domain=(0, 200))), 
                  y = Y('commit_message_length_count')
)

Chart({
  data:      commit_message_length  commit_message_length_count
  0                     16.0                      1372101
  1                     37.0                      1087326
  2                     26.0                       969916
  3                     22.0                       959003
  4                     23.0                       923422
  5                     29.0                       910615
  6                     34.0                       909333
  7                     20.0                       908799
  8                     30.0                       907506
  9                     17.0                       902384
  10                    27.0                       901175
  11                    28.0                       900889
  12                    24.0                       892644
  13                    21.0                       889936
  14                    15.0                       888921
  15                    25.0                       885424
  16                    19.0                       885138
  17                    14.0                       877051
  18                    31.0                       873641
  19                    18.0                       870603
  20                    32.0                       864835
  21                    33.0                       855876
  22                    35.0                       833823
  23                    36.0                       815732
  24                    38.0                       798148
  25                    39.0                       772100
  26                    40.0                       759245
  27                    41.0                       742499
  28                    42.0                       738807
  29                    45.0                       727273
  ..                     ...                          ...
  970                  966.0                         1017
  971                  947.0                         1015
  972                  977.0                         1014
  973                  961.0                         1013
  974                  969.0                         1010
  975                  962.0                         1010
  976                  968.0                         1000
  977                 1098.0                          999
  978                  980.0                          999
  979                  960.0                          991
  980                  989.0                          987
  981                  967.0                          987
  982                  987.0                          982
  983                  986.0                          974
  984                  985.0                          974
  985                 1002.0                          969
  986                  979.0                          969
  987                  972.0                          968
  988                  984.0                          964
  989                  973.0                          956
  990                  974.0                          953
  991                 1020.0                          949
  992                  982.0                          946
  993                 1003.0                          945
  994                  995.0                          941
  995                  993.0                          941
  996                 1028.0                          940
  997                 1001.0                          940
  998                  992.0                          939
  999                 1032.0                          936
  
  [1000 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      scale: Scale({
        domain: (0, 200)
      }),
      shorthand: 'commit_message_length'
    }),
    y: Y({
      shorthand: 'commit_message_length_count'
    })
  }),
  mark: MarkDef({
    clip: True,
    type: 'bar'
  })
})

### g) In a single paragraph (no more than 100 words), describe and analyze the charts you just generated.  What are some interesting trends in the data?  Was there anything that you did not expect? (3 points)

---
*  Majority of the contributors and authors haven't commited more than 10 times. 
*  There are more number of open source web-apps and this can be conveyed with the number of repos using HTML and CSS.
*  Most repos use JavaScript. And it is used for both web-apps and also desktop apps.
*  Most of the length of the commit messages lie between 16 to 40.
*  Most of the repos have < 100 files.
*  Repos use MIT and Apache Licenses.

---

## What Makes a Good Repo?

Given that we have some interesting data at our disposal, let's try to answer the question: what makes a good GitHub repo? For our purposes, a "good" repo is simply a repo with a high watch count; this refers to how many people are following the repo for updates.

To begin, let's see if any of the features *we've already explored* give us any good answers.

## Question 6: Using What We've Worked With (17 points)

Create plots for the following features in a repo and how they relate to that repo's watch count:

1.  License type
2.  Languages
3.  Average file size in a repo
4.  File count in a repo
5.  The number of "power" committers or authors a repo has
6. Average message length of commits on a repo
7. Total Commits in a repo


### a) License type (2 points)

In [0]:
%%bigquery --project my-project-28530 license_type_avg_repo_watch_count

SELECT license, AVG(watch_count) as repo_watch_count
FROM `cs145-ta.project_part2.github_repos_cs145` repositories
JOIN `cs145-ta.project_part2.github_repo_licenses_cs145` repo_licenses
ON repositories. lrepo_name = repo_licenses. lrepo_name
GROUP BY license
ORDER BY repo_watch_count

,license,repo_watch_count
0,epl-1.0,102.212441
1,artistic-2.0,110.666667
2,gpl-2.0,120.958838
3,lgpl-3.0,133.444823
4,isc,134.072553
5,lgpl-2.1,147.007500
6,gpl-3.0,153.551523
7,unlicense,153.594793
8,agpl-3.0,173.819419
9,bsd-2-clause,179.252438


In [0]:
# YOUR PLOT CODE HERE

Chart(license_type_avg_repo_watch_count).mark_bar().encode(
                  x = X('repo_watch_count'), 
                  y = Y('license')
)

Chart({
  data:          license  repo_watch_count
  0        epl-1.0        102.212441
  1   artistic-2.0        110.666667
  2        gpl-2.0        120.958838
  3       lgpl-3.0        133.444823
  4            isc        134.072553
  5       lgpl-2.1        147.007500
  6        gpl-3.0        153.551523
  7      unlicense        153.594793
  8       agpl-3.0        173.819419
  9   bsd-2-clause        179.252438
  10  bsd-3-clause        206.054184
  11           mit        216.790534
  12       mpl-2.0        218.125313
  13    apache-2.0        271.957415
  14       cc0-1.0        475.604854,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'repo_watch_count'
    }),
    y: Y({
      shorthand: 'license'
    })
  }),
  mark: 'bar'
})

In [0]:
%%bigquery --project my-project-28530 license_type_avg_repo_watch_count_popular_repo

SELECT license, AVG(watch_count) as repo_watch_count
FROM `cs145-ta.project_part2.github_repos_cs145` repositories
JOIN `cs145-ta.project_part2.github_repo_licenses_cs145` repo_licenses
ON repositories. lrepo_name = repo_licenses. lrepo_name
WHERE watch_count > 10000
GROUP BY license
ORDER BY repo_watch_count

,license,repo_watch_count
0,mpl-2.0,11664.500000
1,agpl-3.0,13201.500000
2,gpl-2.0,15960.500000
3,apache-2.0,17845.093333
4,mit,18156.688889
5,gpl-3.0,18603.583333
6,bsd-2-clause,19084.000000
7,lgpl-3.0,21399.000000
8,cc0-1.0,24320.250000
9,unlicense,31298.000000


In [0]:
# YOUR PLOT CODE HERE

import altair as alt
import pandas as pd

Chart(license_type_avg_repo_watch_count_popular_repo).mark_bar().encode(
                  x = X('repo_watch_count'), 
                  y = Y('license')
)

Chart({
  data:          license  repo_watch_count
  0        mpl-2.0      11664.500000
  1       agpl-3.0      13201.500000
  2        gpl-2.0      15960.500000
  3     apache-2.0      17845.093333
  4            mit      18156.688889
  5        gpl-3.0      18603.583333
  6   bsd-2-clause      19084.000000
  7       lgpl-3.0      21399.000000
  8        cc0-1.0      24320.250000
  9      unlicense      31298.000000
  10  bsd-3-clause      40744.727273,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'repo_watch_count'
    }),
    y: Y({
      shorthand: 'license'
    })
  }),
  mark: 'bar'
})

### b) Languages (2 points)



In [0]:
%%bigquery --project my-project-28530 repo_language_watch_count

SELECT repo_language, AVG(watch_count) as repo_watch_count
FROM (SELECT repositories. lrepo_name, repo_languages. repo_language, repositories. watch_count 
      FROM (SELECT lrepo_name, language_unnested.name as repo_language
            FROM `cs145-ta.project_part2.github_repo_languages_cs145`
            CROSS JOIN UNNEST(language) as language_unnested) repo_languages 
      JOIN `cs145-ta.project_part2.github_repos_cs145` repositories
      ON repositories. lrepo_name = repo_languages. lrepo_name)
GROUP BY repo_language
ORDER BY repo_watch_count DESC
LIMIT 30

,repo_language,repo_watch_count
0,PureBasic,2848.620000
1,Rascal,2509.727273
2,HiveQL,2128.428571
3,LFE,2124.000000
4,Solidity,2114.611111
5,WebIDL,1854.857143
6,Logtalk,1829.666667
7,IGOR Pro,1817.666667
8,TLA,1705.166667
9,Ballerina,1586.500000


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import pandas as pd

Chart(repo_language_watch_count).mark_bar().encode(
                  x = X('repo_watch_count'), 
                  y = Y('repo_language')
)

Chart({
  data:    repo_language  repo_watch_count
  0      PureBasic       2848.620000
  1         Rascal       2509.727273
  2         HiveQL       2128.428571
  3            LFE       2124.000000
  4       Solidity       2114.611111
  5         WebIDL       1854.857143
  6        Logtalk       1829.666667
  7       IGOR Pro       1817.666667
  8            TLA       1705.166667
  9      Ballerina       1586.500000
  10           Boo       1581.461538
  11          LLVM       1514.592920
  12         Genie       1484.000000
  13   WebAssembly       1440.900000
  14         Smali       1318.625000
  15        Chapel       1176.000000
  16         Fancy        952.666667
  17             q        929.421053
  18   Cap'n Proto        867.828571
  19           Opa        842.000000
  20   AngelScript        831.515625
  21    Dockerfile        741.264727
  22            P4        701.777778
  23           IDL        670.962025
  24     Brainfuck        659.733333
  25       Nearley        658.000000
  26        DTrace        646.560976
  27          Lean        639.785714
  28         COBOL        634.466667
  29           Vue        623.528718,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'repo_watch_count'
    }),
    y: Y({
      shorthand: 'repo_language'
    })
  }),
  mark: 'bar'
})

In [0]:
%%bigquery --project my-project-28530 popular_repo_language_watch_count

SELECT repo_language, AVG(watch_count) as repo_watch_count
FROM (SELECT repositories. lrepo_name, repo_languages. repo_language, repositories. watch_count 
      FROM (SELECT lrepo_name, language_unnested.name as repo_language
            FROM `cs145-ta.project_part2.github_repo_languages_cs145`
            CROSS JOIN UNNEST(language) as language_unnested) repo_languages 
      JOIN `cs145-ta.project_part2.github_repos_cs145` repositories
      ON repositories. lrepo_name = repo_languages. lrepo_name
      WHERE watch_count >= 20000)
GROUP BY repo_language
ORDER BY repo_watch_count DESC
LIMIT 30

,repo_language,repo_watch_count
0,PureBasic,107721.000000
1,Jupyter Notebook,76319.500000
2,CMake,68222.000000
3,LLVM,68222.000000
4,Smarty,67834.500000
5,Rust,57390.000000
6,F#,57390.000000
7,R,57390.000000
8,Clojure,57390.000000
9,Perl 6,57390.000000


In [0]:
# YOUR PLOT CODE HERE

Chart(popular_repo_language_watch_count).mark_bar().encode(
                  x = X('repo_watch_count'), 
                  y = Y('repo_language')
)

Chart({
  data:        repo_language  repo_watch_count
  0          PureBasic     107721.000000
  1   Jupyter Notebook      76319.500000
  2              CMake      68222.000000
  3               LLVM      68222.000000
  4             Smarty      67834.500000
  5               Rust      57390.000000
  6                 F#      57390.000000
  7                  R      57390.000000
  8            Clojure      57390.000000
  9             Perl 6      57390.000000
  10         ShaderLab      57390.000000
  11        Inno Setup      57390.000000
  12              HLSL      57390.000000
  13               PHP      56434.750000
  14      CoffeeScript      54734.000000
  15                C#      53292.000000
  16     Objective-C++      52648.250000
  17              Perl      49860.200000
  18               C++      49858.700000
  19               Vue      49295.250000
  20       Objective-C      48518.000000
  21                 C      46407.272727
  22          Assembly      46123.000000
  23              Ruby      45257.000000
  24               Lua      44976.500000
  25       AppleScript      44224.000000
  26                Go      43003.571429
  27               CSS      41603.567568
  28              HTML      41175.162791
  29        PowerShell      41103.500000,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'repo_watch_count'
    }),
    y: Y({
      shorthand: 'repo_language'
    })
  }),
  mark: 'bar'
})

In [0]:
%%bigquery --project my-project-28530 language_popular_repo_count

SELECT repo_language, COUNT(lrepo_name) as count_repos
FROM (SELECT repositories. lrepo_name, repo_languages. repo_language 
      FROM (SELECT lrepo_name, language_unnested.name as repo_language
            FROM `cs145-ta.project_part2.github_repo_languages_cs145`
            CROSS JOIN UNNEST(language) as language_unnested) repo_languages 
      JOIN `cs145-ta.project_part2.github_repos_cs145` repositories
      ON repositories. lrepo_name = repo_languages. lrepo_name
      WHERE watch_count >= 10000)
GROUP BY repo_language
ORDER BY count_repos DESC
LIMIT 30

,repo_language,count_repos
0,JavaScript,175
1,Shell,151
2,HTML,135
3,CSS,122
4,Makefile,63
5,Python,62
6,Dockerfile,51
7,Ruby,45
8,Java,42
9,TypeScript,40


In [0]:
# YOUR PLOT CODE HERE

Chart(language_popular_repo_count).mark_bar().encode(
                  x = X('count_repos'), 
                  y = Y('repo_language')
)

Chart({
  data:    repo_language  count_repos
  0     JavaScript          175
  1          Shell          151
  2           HTML          135
  3            CSS          122
  4       Makefile           63
  5         Python           62
  6     Dockerfile           51
  7           Ruby           45
  8           Java           42
  9     TypeScript           40
  10             C           37
  11            Go           32
  12     Batchfile           31
  13           C++           29
  14   Objective-C           28
  15         Swift           21
  16    PowerShell           13
  17           Vue           13
  18           PHP           12
  19  CoffeeScript           11
  20            C#           10
  21        Kotlin           10
  22          Roff            9
  23          Perl            9
  24        Groovy            9
  25        Smarty            8
  26         CMake            8
  27           Lua            7
  28      Assembly            7
  29    Vim script            6,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'count_repos'
    }),
    y: Y({
      shorthand: 'repo_language'
    })
  }),
  mark: 'bar'
})

### c) Average ReadMe file size in a repo (2 points)
**Note:** For this question, you may use the `github_repo_readme_contents_cs145` table instead of the full contents table.


In [0]:
%%bigquery --project my-project-28530 file_size_watch_count_distribution

SELECT ceiled_size, AVG(watch_count) as avg_watch_count
FROM (SELECT ceiled_size, watch_count
  	  FROM (SELECT lrepo_name, ROUND(AVG(size) / 1000) as ceiled_size
            FROM (SELECT DISTINCT lrepo_name, id
                  FROM `cs145-ta.project_part2.github_repo_files_cs145`) repo_files
            JOIN `cs145-ta.project_part2.github_repo_readme_contents_cs145` repo_readme
            ON repo_readme. id = repo_files. id
            GROUP BY lrepo_name) repo_size
	    JOIN `cs145-ta.project_part2.github_repos_cs145` repos
	    ON repos. lrepo_name = repo_size. lrepo_name)
WHERE ceiled_size <= 2000
GROUP BY ceiled_size
ORDER BY avg_watch_count

,ceiled_size,avg_watch_count
0,212.0,13.000000
1,640.0,13.000000
2,585.0,13.000000
3,1102.0,13.000000
4,1500.0,13.000000
5,249.0,14.000000
6,146.0,14.000000
7,520.0,14.000000
8,339.0,14.500000
9,479.0,15.000000


In [0]:
# YOUR PLOT CODE HERE

Chart(file_size_watch_count_distribution).mark_bar().encode(
                  x = X('ceiled_size'), 
                  y = Y('avg_watch_count')
)

Chart({
  data:      ceiled_size  avg_watch_count
  0          212.0        13.000000
  1          640.0        13.000000
  2          585.0        13.000000
  3         1102.0        13.000000
  4         1500.0        13.000000
  5          249.0        14.000000
  6          146.0        14.000000
  7          520.0        14.000000
  8          339.0        14.500000
  9          479.0        15.000000
  10         175.0        15.000000
  11         197.0        15.000000
  12         402.0        15.000000
  13         467.0        15.000000
  14        1695.0        16.000000
  15         785.0        17.000000
  16        1098.0        17.000000
  17         584.0        17.000000
  18         245.0        17.000000
  19         269.0        18.000000
  20         761.0        18.000000
  21        1438.0        18.000000
  22        1542.0        18.000000
  23         124.0        18.000000
  24         217.0        18.000000
  25         205.0        18.500000
  26         629.0        19.000000
  27         396.0        19.000000
  28         167.0        19.000000
  29        1176.0        19.000000
  ..           ...              ...
  281         69.0      1836.785714
  282         76.0      1872.666667
  283        134.0      1888.000000
  284         96.0      1894.083333
  285        272.0      1975.000000
  286         44.0      1982.086957
  287         87.0      2030.000000
  288         64.0      2115.375000
  289        155.0      2341.000000
  290        118.0      2369.600000
  291         56.0      2385.687500
  292        974.0      2482.500000
  293        149.0      2597.500000
  294         72.0      2622.300000
  295         70.0      2624.100000
  296        749.0      2767.000000
  297        138.0      3007.000000
  298        261.0      3267.000000
  299         65.0      3425.789474
  300        110.0      3797.333333
  301        191.0      4753.000000
  302        100.0      5357.600000
  303        188.0      5703.000000
  304        113.0      5861.000000
  305        287.0      7170.000000
  306         62.0      7786.800000
  307        463.0      8095.000000
  308        185.0      9126.666667
  309        122.0     19870.000000
  310        360.0     21377.000000
  
  [311 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'ceiled_size'
    }),
    y: Y({
      shorthand: 'avg_watch_count'
    })
  }),
  mark: 'bar'
})

### d) File count in a repo (2 points)

In [0]:
%%bigquery --project my-project-28530 file_size_distribution

SELECT file_count_divided_by_1000, AVG(watch_count) as watch_cnt
FROM (SELECT CEILING(file_count / 1000) as file_count_divided_by_1000, watch_count
      FROM (SELECT lrepo_name, COUNT(id) as file_count
            FROM `cs145-ta.project_part2.github_repo_files_cs145`
            GROUP BY lrepo_name) github_repository_files
      JOIN `cs145-ta.project_part2.github_repos_cs145` github_repositories
      ON github_repository_files. lrepo_name = github_repositories. lrepo_name)
GROUP BY file_count_divided_by_1000

,file_count_divided_by_1000,watch_cnt
0,1.0,193.881320
1,14.0,2366.089286
2,4.0,569.317559
3,3.0,565.505289
4,2.0,498.099862
5,7.0,565.610329
6,51.0,6934.800000
7,5.0,611.843348
8,18.0,2019.809524
9,16.0,805.854167


In [0]:
# YOUR PLOT CODE HERE

Chart(file_size_distribution).mark_bar().encode(
                  x = X('file_count_divided_by_1000',
                      scale=alt.Scale(domain=(1, 500))), 
                  y = Y('watch_cnt')
)

Chart({
  data:      file_count_divided_by_1000    watch_cnt
  0                           1.0   193.881320
  1                          14.0  2366.089286
  2                           4.0   569.317559
  3                           3.0   565.505289
  4                           2.0   498.099862
  5                           7.0   565.610329
  6                          51.0  6934.800000
  7                           5.0   611.843348
  8                          18.0  2019.809524
  9                          16.0   805.854167
  10                          6.0   695.446809
  11                         12.0   662.407895
  12                          8.0   649.051136
  13                         15.0  1216.140000
  14                         91.0  8905.500000
  15                         10.0   741.989130
  16                          9.0   398.093750
  17                         78.0  8271.000000
  18                         13.0   284.215686
  19                         11.0   446.887324
  20                         68.0  5150.000000
  21                        103.0  4570.000000
  22                         44.0   715.500000
  23                       4871.0  3773.000000
  24                         33.0   509.700000
  25                         21.0   582.750000
  26                         60.0  1155.500000
  27                         93.0  1038.000000
  28                         32.0   289.615385
  29                         24.0   318.875000
  ..                          ...          ...
  72                         49.0    23.375000
  73                         30.0    21.333333
  74                         52.0    27.000000
  75                         56.0    29.250000
  76                         88.0    23.000000
  77                        203.0    37.000000
  78                         98.0    24.000000
  79                         84.0    29.000000
  80                       1079.0    30.000000
  81                         97.0    26.000000
  82                         81.0    25.000000
  83                        142.0    25.000000
  84                         77.0    25.000000
  85                         92.0    24.000000
  86                         46.0    16.333333
  87                         99.0    17.500000
  88                         64.0    19.000000
  89                         38.0    17.500000
  90                         62.0    19.000000
  91                         61.0    18.500000
  92                         89.0    19.000000
  93                        185.0    18.000000
  94                         96.0    17.000000
  95                        115.0    17.000000
  96                         70.0    16.000000
  97                         76.0    14.000000
  98                        110.0    13.000000
  99                        362.0    13.000000
  100                      2219.0    13.000000
  101                        39.0    12.000000
  
  [102 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      scale: Scale({
        domain: (1, 500)
      }),
      shorthand: 'file_count_divided_by_1000'
    }),
    y: Y({
      shorthand: 'watch_cnt'
    })
  }),
  mark: 'bar'
})

### e) The number of "power" committers or authors a repo has (3 points)

 **Definition:** a "power" committer or author is an account that has committed/authored 1,000 or more commits.

In [0]:
%%bigquery --project my-project-28530 avg_watch_count_total_power_committers

SELECT total_power_committers, AVG(watch_count) as avg_watch_count
FROM (SELECT total_power_committers, watch_count
      FROM (SELECT lrepo_name, COUNT(email) as total_power_committers
            FROM (SELECT unique_repo_commits. email, unique_repo_commits. lrepo_name, COUNT(unique_repo_commits.commit) as comitter_commit_count
                  FROM (SELECT DISTINCT commit, lrepo_name, committer.email 
                        FROM `cs145-ta.project_part2.github_repo_commits_cs145`) unique_repo_commits
                  GROUP BY lrepo_name, email
                  HAVING comitter_commit_count >= 1000)
            GROUP BY lrepo_name
            ORDER BY total_power_committers DESC) repo_total_power_committers
      JOIN `cs145-ta.project_part2.github_repos_cs145` repos
      ON repos. lrepo_name = repo_total_power_committers. lrepo_name)
GROUP BY total_power_committers

,total_power_committers,avg_watch_count
0,4,3263.313830
1,1,743.468884
2,3,1594.869318
3,11,3876.277778
4,7,2249.291667
5,2,997.680038
6,9,2614.850000
7,8,2915.193548
8,5,1538.226804
9,16,6609.200000


In [0]:
# YOUR PLOT CODE HERE

Chart(avg_watch_count_total_power_committers).mark_bar().encode(
                  x = X('total_power_committers'), 
                  y = Y('avg_watch_count')
)

Chart({
  data:     total_power_committers  avg_watch_count
  0                        4      3263.313830
  1                        1       743.468884
  2                        3      1594.869318
  3                       11      3876.277778
  4                        7      2249.291667
  5                        2       997.680038
  6                        9      2614.850000
  7                        8      2915.193548
  8                        5      1538.226804
  9                       16      6609.200000
  10                      10      1314.480000
  11                      14      2984.333333
  12                       6       935.756410
  13                      12      1076.400000
  14                      19      3464.666667
  15                      13      1457.400000
  16                      18      2097.333333
  17                      56      4570.000000
  18                      25      1154.250000
  19                      21       497.750000
  20                      17       287.000000
  21                     151       932.000000
  22                     124       235.333333
  23                      28       317.000000
  24                      22       146.500000
  25                     105       127.000000
  26                     117        45.000000
  27                      27       116.000000
  28                      76        38.600000
  29                      32        45.714286
  ..                     ...              ...
  33                      34        45.000000
  34                     126        63.000000
  35                      59        30.166667
  36                     102        55.000000
  37                     128        34.500000
  38                      97        25.666667
  39                      61        23.714286
  40                      58        40.000000
  41                     127        39.000000
  42                      50        29.500000
  43                     120        39.000000
  44                      98        35.000000
  45                      92        32.000000
  46                      20        30.000000
  47                      60        22.000000
  48                      52        28.000000
  49                     115        26.000000
  50                      74        18.000000
  51                      70        22.000000
  52                      77        21.666667
  53                     103        21.000000
  54                     116        21.000000
  55                      83        15.000000
  56                      23        16.000000
  57                      78        15.000000
  58                     101        14.000000
  59                      51        14.000000
  60                      80        14.000000
  61                      84        13.000000
  62                      86        13.000000
  
  [63 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'total_power_committers'
    }),
    y: Y({
      shorthand: 'avg_watch_count'
    })
  }),
  mark: 'bar'
})

### f) Average message length of commits on a repo. (3 points)

First make a plot of the average commit length of repositories against the number of repositories with that average commit length. Then, make a plot of how average commit length on a repository correlates to its watch count. Round to nearest byte.


In [0]:
%%bigquery --project my-project-28530 file_size_distribution

SELECT commit_message_length, COUNT(commit_message_length) as number_commit_commit_message_length
FROM (SELECT length(message) as commit_message_length
      FROM `cs145-ta.project_part2.github_repo_commits_cs145`)
GROUP BY commit_message_length
ORDER BY number_commit_commit_message_length DESC
LIMIT 1000

,commit_message_length,number_commit_commit_message_length
0,16,1523629
1,37,1234930
2,26,1157152
3,22,1100579
4,34,1064920
5,29,1060678
6,30,1060328
7,23,1054256
8,28,1046268
9,27,1043998


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import altair as alt
import pandas as pd

Chart(file_size_distribution).mark_bar().encode(
                  x = X('commit_message_length'),
                  y = Y('number_commit_commit_message_length')
)

Chart({
  data:      commit_message_length  number_commit_commit_message_length
  0                       16                              1523629
  1                       37                              1234930
  2                       26                              1157152
  3                       22                              1100579
  4                       34                              1064920
  5                       29                              1060678
  6                       30                              1060328
  7                       23                              1054256
  8                       28                              1046268
  9                       27                              1043998
  10                      20                              1032404
  11                      17                              1028692
  12                      24                              1028399
  13                      31                              1025833
  14                      25                              1019050
  15                      32                              1015585
  16                      21                              1012585
  17                      33                              1010171
  18                      19                              1003625
  19                      15                               994098
  20                      14                               987543
  21                      35                               986864
  22                      18                               986728
  23                      36                               975855
  24                      38                               961558
  25                      39                               933025
  26                      40                               921868
  27                      42                               893593
  28                      41                               891271
  29                      45                               877354
  ..                     ...                                  ...
  970                    953                                 9678
  971                    936                                 9579
  972                   1002                                 9557
  973                   1009                                 9547
  974                   1041                                 9538
  975                    984                                 9492
  976                    956                                 9392
  977                    931                                 9369
  978                    987                                 9336
  979                    916                                 9317
  980                   1023                                 9315
  981                    998                                 9313
  982                    944                                 9276
  983                    939                                 9275
  984                    994                                 9268
  985                   1008                                 9263
  986                    992                                 9224
  987                   1015                                 9216
  988                   1005                                 9185
  989                    991                                 9158
  990                    980                                 9149
  991                    982                                 9117
  992                    995                                 9082
  993                    968                                 9003
  994                    974                                 8992
  995                    985                                 8979
  996                    961                                 8963
  997                    963                                 8943
  998                    9

In [0]:
%%bigquery --project my-project-28530 watch_count_commit_message_length_distribution

SELECT rounded_avg_commit_message_length, AVG(watch_count) as avg_watch_count
FROM (SELECT ROUND(avg_commit_message_length) as rounded_avg_commit_message_length, watch_count
      FROM (SELECT lrepo_name, AVG(message_length) as avg_commit_message_length
            FROM (SELECT lrepo_name, length(message) as message_length
                  FROM `cs145-ta.project_part2.github_repo_commits_cs145`)
            GROUP BY lrepo_name) repo_avg_commit_message_length
      JOIN `cs145-ta.project_part2.github_repos_cs145` repos
      ON repos. lrepo_name = repo_avg_commit_message_length. lrepo_name)
WHERE rounded_avg_commit_message_length <= 1000
GROUP BY rounded_avg_commit_message_length

,rounded_avg_commit_message_length,avg_watch_count
0,76.0,649.588308
1,54.0,322.352864
2,168.0,2571.239130
3,153.0,1701.923077
4,52.0,226.769762
5,84.0,358.490018
6,74.0,406.469478
7,95.0,482.290909
8,116.0,651.613095
9,65.0,305.599719


In [0]:
# YOUR PLOT CODE HERE

Chart(watch_count_commit_message_length_distribution).mark_bar().encode(
                  x = X('rounded_avg_commit_message_length'),
                  y = Y('avg_watch_count')
)

Chart({
  data:      rounded_avg_commit_message_length  avg_watch_count
  0                                 76.0       649.588308
  1                                 54.0       322.352864
  2                                168.0      2571.239130
  3                                153.0      1701.923077
  4                                 52.0       226.769762
  5                                 84.0       358.490018
  6                                 74.0       406.469478
  7                                 95.0       482.290909
  8                                116.0       651.613095
  9                                 65.0       305.599719
  10                               123.0       665.597222
  11                               161.0      1111.644068
  12                               134.0       571.073171
  13                                63.0       295.666889
  14                                67.0       394.429017
  15                                87.0       344.156780
  16                                45.0       207.541773
  17                               117.0       643.291667
  18                                80.0       348.339339
  19                                55.0       241.773593
  20                                47.0       240.870651
  21                               234.0      2704.692308
  22                                38.0       164.662531
  23                                57.0       263.839391
  24                                64.0       315.948070
  25                               125.0       408.548387
  26                                59.0       298.357104
  27                               196.0      1277.307692
  28                               233.0      2457.083333
  29                                89.0       358.790254
  ..                                 ...              ...
  566                              663.0        17.000000
  567                              513.0        17.000000
  568                              865.0        16.000000
  569                              417.0        16.000000
  570                              531.0        15.000000
  571                              858.0        16.000000
  572                              529.0        16.000000
  573                              598.0        16.000000
  574                              793.0        16.000000
  575                              998.0        16.000000
  576                              520.0        15.333333
  577                              377.0        15.000000
  578                              754.0        15.000000
  579                              624.0        15.000000
  580                              497.0        15.000000
  581                              367.0        14.000000
  582                              468.0        14.500000
  583                              546.0        14.500000
  584                              528.0        14.500000
  585                              454.0        15.000000
  586                              891.0        15.000000
  587                              461.0        14.000000
  588                              571.0        13.500000
  589                              570.0        13.500000
  590                              357.0        14.000000
  591                              387.0        14.000000
  592                              548.0        13.000000
  593                              338.0        13.000000
  594                              500.0        13.000000
  595                              557.0        13.000000
  
  [596 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'rounded_avg_commit_message_length'
    }),
    y: Y({
      shorthand: 'avg_watch_count'
    })
  }),
  mark: 'bar'
})

### Repo_Frequency vs Total Commits in the repo

In [0]:
%%bigquery --project my-project-28530 repo_frequency_total_repo_commits

SELECT commit_count, COUNT(commit_count) as repo_freq
FROM (SELECT CEILING(COUNT(commit) / 1000) as commit_count
      FROM (SELECT DISTINCT commit, lrepo_name
            FROM `cs145-ta.project_part2.github_repo_commits_cs145`)
      GROUP BY lrepo_name) repo_commit_count
WHERE commit_count <= 1000 AND commit_count > 1
GROUP BY commit_count

,commit_count,repo_freq
0,2.0,6698
1,4.0,1177
2,485.0,1
3,6.0,474
4,231.0,1
5,534.0,1
6,71.0,3
7,15.0,74
8,7.0,328
9,53.0,2


In [0]:
# YOUR PLOT CODE HERE

Chart(repo_frequency_total_repo_commits).mark_bar().encode(
                  x = X('commit_count',
                        scale=alt.Scale(domain=(1, 50)) 
                       ),
                  y = Y('repo_freq')
)

Chart({
  data:      commit_count  repo_freq
  0             2.0       6698
  1             4.0       1177
  2           485.0          1
  3             6.0        474
  4           231.0          1
  5           534.0          1
  6            71.0          3
  7            15.0         74
  8             7.0        328
  9            53.0          2
  10           48.0          5
  11            8.0        245
  12          430.0          2
  13           30.0         15
  14            3.0       2377
  15          606.0          1
  16           12.0         86
  17           10.0        156
  18           27.0         11
  19           16.0         63
  20            9.0        177
  21          129.0          1
  22          331.0          1
  23           21.0         29
  24            5.0        703
  25          136.0          1
  26           98.0          2
  27           37.0          6
  28           55.0          3
  29           13.0         72
  ..            ...        ...
  133         100.0          1
  134         135.0          2
  135         114.0          1
  136         562.0          1
  137         508.0          1
  138         638.0          1
  139         635.0          1
  140         410.0          2
  141         431.0          1
  142         380.0          1
  143         346.0          1
  144         182.0          3
  145         260.0          1
  146         607.0          1
  147         444.0          1
  148          67.0          3
  149         172.0          1
  150          58.0          1
  151         133.0          1
  152         599.0          1
  153         353.0          2
  154         350.0          1
  155         104.0          1
  156         649.0          1
  157          74.0          1
  158         674.0          1
  159         123.0          1
  160         169.0          1
  161         352.0          1
  162          81.0          1
  
  [163 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      scale: Scale({
        domain: (1, 50)
      }),
      shorthand: 'commit_count'
    }),
    y: Y({
      shorthand: 'repo_freq'
    })
  }),
  mark: 'bar'
})

### Watch Count vs Total Commits in the repo

In [0]:
%%bigquery --project my-project-28530 watch_count_total_repo_commits

SELECT AVG(watch_count) as avg_watch_count, commit_count
FROM (SELECT lrepo_name, CEILING(COUNT(commit) / 1000) as commit_count
      FROM (SELECT DISTINCT commit, lrepo_name
            FROM `cs145-ta.project_part2.github_repo_commits_cs145`)
      GROUP BY lrepo_name) repo_commit_count
JOIN `cs145-ta.project_part2.github_repos_cs145` repos
ON repos. lrepo_name = repo_commit_count. lrepo_name
WHERE commit_count <= 1000
GROUP BY commit_count

,avg_watch_count,commit_count
0,161.038818,1.0
1,610.287250,2.0
2,261.000000,67.0
3,354.600000,50.0
4,870.466667,22.0
5,903.824148,3.0
6,4552.933333,36.0
7,39.000000,260.0
8,25.000000,444.0
9,17.000000,607.0


In [0]:
# YOUR PLOT CODE HERE

Chart(watch_count_total_repo_commits).mark_bar().encode(
                  x = X('commit_count'),
                  y = Y('avg_watch_count')
)

Chart({
  data:      avg_watch_count  commit_count
  0         161.038818           1.0
  1         610.287250           2.0
  2         261.000000          67.0
  3         354.600000          50.0
  4         870.466667          22.0
  5         903.824148           3.0
  6        4552.933333          36.0
  7          39.000000         260.0
  8          25.000000         444.0
  9          17.000000         607.0
  10       1864.333333          78.0
  11         96.000000         172.0
  12        713.724359          10.0
  13       2010.375000          39.0
  14       1571.551724          11.0
  15       1430.500000          32.0
  16       1117.539634           7.0
  17        999.004219           6.0
  18       5049.674419          12.0
  19         49.333333         182.0
  20        961.567347           8.0
  21        235.666667         126.0
  22       1117.351351          15.0
  23        105.000000          38.0
  24        941.000000          43.0
  25       1205.706215           9.0
  26        691.453125          14.0
  27        843.409673           5.0
  28        759.432455           4.0
  29       1102.638889          13.0
  ..               ...           ...
  134        71.000000         287.0
  135        14.000000         277.0
  136        68.000000         576.0
  137      1219.000000         101.0
  138        52.000000         665.0
  139        21.000000         520.0
  140        26.000000         381.0
  141        39.000000         618.0
  142        21.000000          99.0
  143        16.000000         379.0
  144        38.000000          47.0
  145       108.000000         106.0
  146        26.000000         148.0
  147       317.000000         105.0
  148        16.000000          94.0
  149        30.000000         344.0
  150        13.000000         564.0
  151        20.000000         378.0
  152        32.000000         469.0
  153        22.000000         362.0
  154        30.000000          87.0
  155        28.000000         354.0
  156        17.000000         664.0
  157        18.000000         308.0
  158        21.000000         103.0
  159        64.000000          96.0
  160        40.000000         315.0
  161       526.000000          59.0
  162       314.000000          73.0
  163      1161.000000          91.0
  
  [164 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'commit_count'
    }),
    y: Y({
      shorthand: 'avg_watch_count'
    })
  }),
  mark: 'bar'
})

### h) Which, if any, of the features we inspected above have the highest correlation with a repo having a high watch count? Does the answer make sense, or does it seem counterintuitive? Explain your answer in a small paragraph, no more than 200 words.  Be sure to cite the charts you generated. (3 points)

---
*  Almost every popular repo uses Java Script. 
*  Popular repos have higher number of commits.
*  Popular Repos have larger Read Me File Size.
*  Repos with high commit message are more popular. (The standard open source Projects).
*  bsd-3-clause	license is used by the popular repos.
*  Repos with higher file count are popular.

---

## What Do Others Have to Say?

At this point we have learned a couple of things about how certain features may or may not impact the popularity of a GitHub repo.  However, we really only looked at features of GitHub repos that we had initially explored when we were getting a feel for the dataset!  There has got to be more things we can inspect than that.

If you do a web search for "how to make my git repo popular," you will find that more than a couple of people suggest investing time in your `README` file.  The `README` usually gives an overview to a GitHub project and may include other information about the codebase such as whether its most recent build passed or how to begin contributing to that repo.  [Here](https://github.com/vuejs/vue/blob/dev/README.md) is an example `README` file for the popular web development framework Vue.js.


## IMPORTANT: Note about Contents Table

Note that the original `github_repo_contents_cs145` table is about half a TB!  In order to save you the pain of using up 500GB of your credits to subset this table into a workable size for this problem, we have done it for you. 

## ***For the rest of this question, be sure that you use the `github_repo_readme_contents_cs145` table ***

## Question 7: Analyzing Features (15 points)

Analyze the following features of a repo's `README` file and how they relate to the popularity of a repository, generating an informative plot for each feature:

1.   Having or not having a `README` file
2.   The length of the `README` file
3.   The number of images (.jpg, .png, .svg) or GIFs (.gif) included in the `README`

Note that this part is intentionally more open ended than usual.  Part of what we are looking for is the thoroughness with which you leverage SQL and your plots to gain insight into these features. 

 **Hint:** BigQuery's SQL string, REGEX, and array functions will be helpful here.
 
 **Hint:** You can develop and test regexes on online testers like https://regex101.com/.

### a) Having or not having a `README` file (4 points)




In [0]:
%%bigquery --project my-project-28530 contains_repo_watch_count

SELECT 'contains_readme' as contains_readme, AVG(watch_count) as watch_count
FROM `cs145-ta.project_part2.github_repos_cs145` repo_popularity
WHERE lrepo_name IN (SELECT DISTINCT lrepo_name
                     FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145` readme_contents 
                     JOIN `cs145-ta.project_part2.github_repo_files_cs145` repo_files
                     ON repo_files. id = readme_contents. id)

UNION ALL                          
      
SELECT 'no readme' as contains_readme, AVG(watch_count) as watch_count
FROM `cs145-ta.project_part2.github_repos_cs145` repo_popularity
WHERE lrepo_name NOT IN (SELECT DISTINCT lrepo_name
                         FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145` readme_contents 
                         JOIN `cs145-ta.project_part2.github_repo_files_cs145` repo_files
                         ON repo_files. id = readme_contents. id)

,contains_readme,watch_count
0,contains_readme,212.726830
1,no readme,118.673641


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import altair as alt
import pandas as pd

Chart(contains_repo_watch_count).mark_bar().encode(
                  x = X('watch_count'),
                  y = Y('contains_readme')
)

Chart({
  data:    contains_readme  watch_count
  0  contains_readme   212.726830
  1        no readme   118.673641,
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'watch_count'
    }),
    y: Y({
      shorthand: 'contains_readme'
    })
  }),
  mark: 'bar'
})

### b) The length of the `README` file (2 points)

You may ignore README files with length 0.  

**Note**: If a project has multiple `README` files, you can just take the average size of those files.

In [0]:
%%bigquery --project my-project-28530 file_size_distribution

# Already Done!

In [0]:
# YOUR PLOT CODE HERE

# Already Done!

### c) The number of images (`.jpg`, `.png`, `.svg`) or GIFs (`.gif`) included in the `README` (5 points)

#### Plot the total count of `.png`, `.jpg`, `.svg`, and `.gif` files included.  Remember you can sample the results on a DataFrame for clearer visualizations.

You can find whether an image/GIF is included in a `README` by searching for the following strings:
*   \!\[{*zero+ characters that are not* ']'}]({*zero+ chars that are not* ')'}.(*png|jpg|svg|gif*))
> Example: `![my_svg](https://foo.io/v/svgs/my_cool_svg.svg)`
* <{*zero+ whitespace*}*img*{*one+ whitespace*}src="{*zero+ chars not *'\"'}.(*png|jpg|svg|gif*)
> Example: `<img src="https://foo.bar/attachments/cat.gif" />`

**Hint**: You should test out your regular expressions on a subset of README files so that you can manually verify they work.

**Note**: If you end up dealing with multiple versions of the same `README.md` file, it's fine to average statistics among them.

In [0]:
%%bigquery --project my-project-28530 repo_watch_image_count

SELECT AVG(avg_watch_count) as avg_repo_watch_count, total_image_count
FROM (SELECT avg_watch_count,  
             avg_jpg_count + avg_png_count + avg_svg_count + avg_gif_count as total_image_count
      FROM (SELECT repos. lrepo_name, 
                   CEILING(AVG(watch_count)) as avg_watch_count, 
                   CEILING(AVG(jpg_count)) as avg_jpg_count, 
                   CEILING(AVG(png_count)) as avg_png_count, 
                   CEILING(AVG(svg_count)) as avg_svg_count, 
                   CEILING(AVG(gif_count)) as avg_gif_count
            FROM (SELECT DISTINCT lrepo_name, jpg_count, png_count, svg_count, gif_count
                  FROM (SELECT id, 
                               (LENGTH(content) - LENGTH(REPLACE(content, '.jpg', ''))) / 4 as jpg_count, 
                               (LENGTH(content) - LENGTH(REPLACE(content, '.png', ''))) / 4 as png_count, 
                               (LENGTH(content) - LENGTH(REPLACE(content, '.svg', ''))) / 4 as svg_count, 
                               (LENGTH(content) - LENGTH(REPLACE(content, '.gif', ''))) / 4 as gif_count
                        FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145`
                        ) repo_files_images
                  JOIN `cs145-ta.project_part2.github_repo_files_cs145` repo_files
                  ON repo_files_images. id = repo_files. id) repo_images
            JOIN `cs145-ta.project_part2.github_repos_cs145` repos
            ON repos. lrepo_name = repo_images. lrepo_name
            GROUP BY repos. lrepo_name))
WHERE total_image_count <= 200
GROUP BY total_image_count

,avg_repo_watch_count,total_image_count
0,455.970573,8.0
1,104.025439,0.0
2,310.097455,3.0
3,170.446438,1.0
4,218.631698,2.0
5,322.918759,4.0
6,330.072578,5.0
7,386.316602,6.0
8,580.040816,10.0
9,681.875000,16.0


In [0]:
# YOUR PLOT CODE HERE

from altair import *
import altair as alt
import pandas as pd

Chart(repo_watch_image_count).mark_bar().encode(
                  x = X('total_image_count'),
                  y = Y('avg_repo_watch_count')
)

Chart({
  data:      avg_repo_watch_count  total_image_count
  0              455.970573                8.0
  1              104.025439                0.0
  2              310.097455                3.0
  3              170.446438                1.0
  4              218.631698                2.0
  5              322.918759                4.0
  6              330.072578                5.0
  7              386.316602                6.0
  8              580.040816               10.0
  9              681.875000               16.0
  10             461.447257                7.0
  11             480.626964                9.0
  12             261.575758               27.0
  13             224.500000               39.0
  14             583.104455               11.0
  15             243.000000              132.0
  16            1207.000000               20.0
  17             995.655172               18.0
  18             560.692308               23.0
  19             593.074803               12.0
  20             741.133333               13.0
  21             772.559880               14.0
  22            1262.819048               19.0
  23             948.587500               17.0
  24            1056.748092               15.0
  25             355.000000               52.0
  26            1133.612903               29.0
  27            1179.770270               21.0
  28            1000.000000              125.0
  29            1104.603774               24.0
  ..                    ...                ...
  90             247.000000               77.0
  91             359.500000               98.0
  92            3502.750000               65.0
  93            1016.000000               62.0
  94             711.000000              107.0
  95             105.000000              129.0
  96            3040.000000               90.0
  97              17.000000              149.0
  98            8052.000000              167.0
  99            5108.000000               58.0
  100           8981.500000              102.0
  101           1324.500000               57.0
  102           1290.000000               74.0
  103            245.333333               59.0
  104            308.000000               93.0
  105             38.000000               96.0
  106            498.500000               80.0
  107           2256.800000               42.0
  108             52.000000              135.0
  109             22.000000              116.0
  110           1572.000000               53.0
  111             48.000000              103.0
  112           3692.000000              185.0
  113            170.000000              131.0
  114            161.000000               75.0
  115            120.500000              111.0
  116            192.000000              101.0
  117             95.000000              177.0
  118             14.000000               78.0
  119           2729.000000              112.0
  
  [120 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'total_image_count'
    }),
    y: Y({
      shorthand: 'avg_repo_watch_count'
    })
  }),
  mark: 'bar'
})

### d) Web Links

In [0]:
%%bigquery --project my-project-28530 repo_watch_links_count

SELECT AVG(avg_watch_count) as avg_repo_watch_count, total_image_count as total_links_count
FROM (SELECT avg_watch_count,  
             avg_jpg_count + avg_png_count as total_image_count
      FROM (SELECT repos. lrepo_name, 
                   CEILING(AVG(watch_count)) as avg_watch_count, 
                   CEILING(AVG(jpg_count)) as avg_jpg_count, 
                   CEILING(AVG(png_count)) as avg_png_count 
            FROM (SELECT DISTINCT lrepo_name, jpg_count, png_count
                  FROM (SELECT id, 
                               (LENGTH(content) - LENGTH(REPLACE(content, 'http://', ''))) / 4 as jpg_count, 
                               (LENGTH(content) - LENGTH(REPLACE(content, 'https://', ''))) / 4 as png_count
                        FROM `cs145-ta.project_part2.github_repo_readme_contents_cs145`
                        ) repo_files_images
                  JOIN `cs145-ta.project_part2.github_repo_files_cs145` repo_files
                  ON repo_files_images. id = repo_files. id) repo_images
            JOIN `cs145-ta.project_part2.github_repos_cs145` repos
            ON repos. lrepo_name = repo_images. lrepo_name
            GROUP BY repos. lrepo_name))
GROUP BY total_image_count

,avg_repo_watch_count,total_links_count
0,318.269912,22.0
1,4454.625000,134.0
2,1423.074380,71.0
3,423.309148,36.0
4,284.384187,30.0
5,55436.000000,654.0
6,2915.227273,160.0
7,180.468350,14.0
8,122.979434,3.0
9,334.561374,32.0


In [0]:
# YOUR PLOT CODE HERE

Chart(repo_watch_links_count).mark_bar().encode(
                  x = X('total_links_count'),
                  y = Y('avg_repo_watch_count')
)

Chart({
  data:      avg_repo_watch_count  total_links_count
  0              318.269912               22.0
  1             4454.625000              134.0
  2             1423.074380               71.0
  3              423.309148               36.0
  4              284.384187               30.0
  5            55436.000000              654.0
  6             2915.227273              160.0
  7              180.468350               14.0
  8              122.979434                3.0
  9              334.561374               32.0
  10            2950.882353              167.0
  11             865.436548               59.0
  12             144.665282               12.0
  13           37658.000000              759.0
  14            2130.642857              118.0
  15            2682.360000              127.0
  16             438.854839               35.0
  17             259.579190               19.0
  18            5337.700000              196.0
  19            1197.690476               98.0
  20             258.481120               28.0
  21            2038.421053              156.0
  22            2025.222222              111.0
  23           32310.000000              930.0
  24            1682.303030               83.0
  25              61.688617                0.0
  26             144.640936               10.0
  27             614.133333               66.0
  28            1176.326531               87.0
  29             271.625450               24.0
  ..                    ...                ...
  592             22.000000             1988.0
  593             22.000000              729.0
  594             21.000000             3236.0
  595             20.000000              342.0
  596             20.000000              556.0
  597             20.000000              917.0
  598             19.000000             3552.0
  599             19.000000              269.0
  600             19.000000             8045.0
  601             19.000000              330.0
  602             18.000000              877.0
  603             18.000000              455.0
  604             18.000000             1704.0
  605             17.000000              476.0
  606             17.000000              725.0
  607             17.000000              901.0
  608             16.000000              922.0
  609             16.000000              380.0
  610             16.000000             1080.0
  611             16.000000              552.0
  612             15.000000              405.0
  613             15.000000              424.0
  614             15.000000              382.0
  615             15.000000             1189.0
  616             14.000000              722.0
  617             14.000000             1813.0
  618             14.000000             1169.0
  619             13.000000              766.0
  620             13.000000              377.0
  621             29.636364                NaN
  
  [622 rows x 2 columns],
  encoding: EncodingWithFacet({
    x: X({
      shorthand: 'total_links_count'
    }),
    y: Y({
      shorthand: 'avg_repo_watch_count'
    })
  }),
  mark: 'bar'
})

### e) Would you say that a "good" `README` is correlated with a popular repository, based on the features you studied?  Why or why not?  If you were to analyze more in-depth features on the `README` file for correlation with repo popularity what would they be? (4 points)

---
* Repos that contain ReadMe are more popular.
* Repos with greater ReadMe size are more popular.
* Providing Images in the README file increases the popularity.
* Including Links in the Repo might increase the popularity of the repo.

---